### Introduction
 * In this chapter, we’ll dissect the key components of a tool use system and what a tool use workflow looks like. And we’ll do that with a concrete code example.
To build an intuition of how tool use works, let’s start with a simple example. We’ll look at a use case of a RAG assistant that can query the sales database of an e-commerce company.
In summary, there are two parts to building a tool use application:
* Setting up the tools
* Running the tool use workflow

In [1]:
# Setting Up the Tools
import json
import cohere
co = cohere.Client(api_key="j3dnErL21HLrrwoaSskJsQpmAn2SxRc1JzJWg7yL")

In [2]:
# Creating the tool (tool can be any function or service that can receive and send data.)
# Defining the tool schema

In [6]:
# Step 1: Create the Tool
def daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    # Mock database containing daily sales reports
    sales_database = {
    '2023-09-28': {'total_sales_amount': 5000,'total_units_sold': 100},
    '2023-09-29': {'total_sales_amount': 10000,'total_units_sold': 250},
    '2023-09-30': {'total_sales_amount': 8000,'total_units_sold': 200}
    }
    report = sales_database.get(day,{})
    if report:
        return{
            'date':day,
            'summary':f"Total Sales Amount:{report['total_sales_amount']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}
    

functions_map = {
    "daily_sales_report": daily_sales_report
}

In [7]:
# Step 2: Define Tool Schema
tools = [
    {
        "name": "daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    }
]

In [8]:
# Step 3: Create Custom Preamble (Optional)
preamble = """## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

### Running the tool workflow
The tool is now ready to use. We can think of a tool use system as consisting of four components:

* The user
* The application
* The LLM
* The tools

In [9]:
# Step 1: Get User Message
message = "Can you provide a sales summary for 29th September 2023?"

In [15]:
# Step 2: Generate Tool Calls
model = "command-a-03-2025"
# Initial response to the user message
response = co.chat(
    message=message,
    model=model,
    preamble=preamble,
    tools=tools,
    force_single_step=True
)

tool_calls = response.tool_calls
    
print("Tool calls:\n")
for i,t in enumerate(tool_calls):
    print(f"#{i+1}\nTool: {t.name}\nParameters: {t.parameters}\n")
    

Tool calls:

#1
Tool: daily_sales_report
Parameters: {'day': '2023-09-29'}



In [16]:
# Step 3: Get Tool Results
tool_results = []
for tc in tool_calls:
    tool_output = functions_map[tc.name](**tc.parameters)
    tool_call = {"name": tc.name, "parameters": tc.parameters}
    tool_results.append({"call": tool_call, "outputs": [tool_output]})
    
print("Tool results:\n")
for i, t in enumerate(tool_results):
    print(f"#{i+1}\nTool call: {t['call']}\nOutputs: {t['outputs']}\n")

Tool results:

#1
Tool call: {'name': 'daily_sales_report', 'parameters': {'day': '2023-09-29'}}
Outputs: [{'date': '2023-09-29', 'summary': 'Total Sales Amount:10000'}]



In [18]:
# Step 4: Generate Response and Citations
# response.chat_history

# Generate response
response = co.chat(
    message="",
    model=model,
    preamble=preamble,
    tools=tools,
    tool_results=tool_results,
    chat_history=response.chat_history
)
    
print("Final response:")
print(response.text)
print("="*50)

if response.citations:
    print("\nCitations:")
    for citation in response.citations:
        print(citation)
    print("\nCited Documents:")
    for document in response.documents:
        print(document)

Final response:
The total sales amount for 29th September 2023 was 10,000.

Citations:
start=51 end=58 text='10,000.' document_ids=['daily_sales_report:0:2:0'] type='TEXT_CONTENT'

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount:10000'}


In [21]:
# Streaming the Response
 # Generate response
response = co.chat_stream(
    message="",
    model=model,
    preamble=preamble,
    tools=tools,
    tool_results=tool_results,
    chat_history=response.chat_history
)
    
print("Final response:\n")
chatbot_response = ""

for event in response:
    if event.event_type == "text-generation":
        print(event.text, end="")
        chatbot_response += event.text
    if event.event_type == "stream-end":
        if event.response.citations:
            print("\n\nCitations:")
            for citation in event.response.citations:
                print(citation)
        if event.response.documents:
            print("\nCited Documents")
            for document in event.response.documents:
                print(document)

Final response:

The total sales amount for 29th September 2023 was 10,000.

Citations:
start=51 end=58 text='10,000.' document_ids=['daily_sales_report:0:4:0'] type='TEXT_CONTENT'

Cited Documents
{'date': '2023-09-29', 'id': 'daily_sales_report:0:4:0', 'summary': 'Total Sales Amount:10000'}


## Tool Use Modes
 * A good tool use API must be able to handle different scenarios, from simple to complex LLM tool calls. This translates to being able to call tools in different modes: single-step, multi-step, and parallel.

In [22]:
# Single-Step
# Single-Step, Parallel
#  calling tools in parallel. This can be calling the same tool multiple times or different tools for any number of calls.


# Multi-Step
# output of one tool is needed as the input to the other tool.
# Multi-Step, Parallel
